In [ ]:
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("darkgrid")

efficiency_csp_power_block = 0.412 # from data/efficiencies.csv

nice_names = {
    "csp-tower": "CSP",
    "csp-tower power block": "CSP",
    "solar-utility": "PV",
    "onwind": "Wind onshore",
    "offwind": "Wind offshore",
}

nice_colors = {
    "CSP": "orange",
    "PV": "#f9d002",
    "Wind onshore": "#235ebc",
    "Wind offshore": "#6895dd",
}

# Select data to plot
df = pd.read_csv(snakemake.input["results"], delimiter=";")
df = df.loc[
    (df["exporter"] == snakemake.wildcards["exporter"]) & 
    (df["esc"] == snakemake.wildcards["esc"])]

df["flexibility"] = df["scenario"].str.replace("_.*", "", regex=True)
df["csp"] = df["scenario"].str.contains("with_csp")

# Add sorted category column to df indicating type of flexibility
# then sort by that category to ensure correct order in x-axis of plot
# Note: Reproduces the order given in "flexibilities".

flexibilities = {
    "unbuffered": "No flexibility",    
    "daily"     : "Daily",
    "weekly"    : "Weekly",
    "biweekly"  : "Biweekly",
    "monthly"   : "Month",
    "quaterly"  : "Quaterly",
    "annually"  : "Yearly",
}

df["flexibility"] = df.flexibility.replace(flexibilities)

df["flexibility"] = pd.Categorical(
    df.flexibility,
    categories=flexibilities.values(),
    ordered=True)

df = df.sort_values("flexibility")

In [ ]:
# Restrict to only needed information from now on
df = df.query("category == 'RES'")
df = df.drop(columns=["year","wacc","esc","exporter","importer","category","scenario"])

technologies = ["onwind", "offwind", "solar-utility", "csp-tower power block", "csp-tower"]

electricity_columns = [f"Total generated electricity from {t}" for t in technologies]
capacity_columns = [f"Total installed capacity {t}" for t in technologies]

electricity = df.loc[df["subcategory"].isin(electricity_columns)]
electricity["subcategory"] = electricity["subcategory"].str.replace("Total generated electricity from ","")
electricity = electricity.pivot_table(index=["csp", "flexibility"], columns="subcategory", values="value")

capacity = df.loc[df["subcategory"].isin(capacity_columns)]
capacity["subcategory"] = capacity["subcategory"].str.replace("Total installed capacity ","")
capacity = capacity.pivot_table(index=["csp","flexibility"], columns="subcategory", values="value")
capacity["csp-tower"] *= efficiency_csp_power_block # Weigh CSP capacity with heat-to-electricity efficiency of CSP


electricity /= 1e6 # in TWh
capacity /= 1e3 # in GW

# Rename to nicer names for plot legend
electricity = electricity.rename(columns={c:nice_names[c] for c in electricity.columns})
capacity = capacity.rename(columns={c:nice_names[c] for c in capacity.columns})

# Reorder for alphabetical legend order
electricity = electricity[sorted(electricity.columns)]
capacity = capacity[sorted(capacity.columns)]

with open(snakemake.log[0], "w") as f:
    print("Capacity difference in GW (highest/lowest capacity of all flexibilities)", file=f)
    print(capacity.sum(axis="columns").groupby("csp").agg([
        np.ptp,lambda x: np.round(np.ptp(x)/np.max(x),4)*100]
    ), file=f)

fig, axes = plt.subplots(2,2,figsize=(8,5), sharex=True, sharey="row")

capacity.loc[False,].plot.bar(ax=axes[0][0], stacked=True, color=[nice_colors[c] for c in capacity.columns])
capacity.loc[True,].plot.bar(ax=axes[0][1], stacked=True, color=[nice_colors[c] for c in capacity.columns])

electricity.loc[False,].plot.bar(ax=axes[1][0], stacked=True, color=[nice_colors[c] for c in electricity.columns])
electricity.loc[True,].plot.bar(ax=axes[1][1], stacked=True, color=[nice_colors[c] for c in electricity.columns])

axes[0][0].set_title("Without CSP")
axes[0][1].set_title("With CSP")

axes[0][0].set_ylabel("Installed Capacity \n[GW]")
axes[1][0].set_ylabel("Annual Generation \n[TWh]")
axes[1][0].set_xlabel("Flexibility to shift load")
axes[1][1].set_xlabel("Flexibility to shift load")

axes[0][0].legend(title=False, ncol=2)
axes[0][1].legend().set_visible(False)
axes[1][0].legend().set_visible(False)
axes[1][1].legend().set_visible(False)

# Set upper value of y-axis to next 10er number
# Set upper value of y-axis to 90 as magic number across all ESCs
axes[0][0].set_ylim(0, 90)#np.ceil(capacity.sum(axis="columns").max()/10)*10)
axes[1][0].set_ylim(0, 90)#np.ceil(electricity.sum(axis="columns").max()/10)*10)

[label.set_rotation(60) for label in axes[1][0].get_xticklabels()]
[label.set_rotation(60) for label in axes[1][1].get_xticklabels()]

fig.tight_layout()

# Saving figure
fig.savefig(snakemake.output["pdf"], dpi=300, bbox_inches='tight')
fig.savefig(snakemake.output["png"], dpi=300, bbox_inches='tight')